In [3]:
import os
import time
import numpy as np
# from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
# from sklearn.metrics import log_loss
import random as rn
# from models.dcsn3d import DCSNMain as DCSN
from models.dcsn2d_light import Teacher , Student
# from models.dcsn2d_light_MSI_awgn import DCSN_Teacher , DCSN_Student
from utils import *
import torch
import numpy as np
from dataset import *
from scipy.io import savemat, loadmat
from trainOps import *
import tqdm
import time
import argparse

# torch.backends.cudnn.benchmark=True
# Hyperparameters
batch_size = 1
device = 'cuda'
MAX_EP = 16000
BANDS = 172
SIGMA = 0.0    ## Noise free -> SIGMA = 0.0

prefix='DCSN_joint2'
DEBUG=False


def parse_args():
    parser = argparse.ArgumentParser(description='Train Convex-Optimization-Aware SR net')
    
    parser.add_argument('--SEED', type=int, default=1029)
    parser.add_argument('--batch_size', type=int, default=1)

    parser.add_argument('--resume_ind', type=int, default=999)
    parser.add_argument('--snr', type=int, default=0)
    
    
    parser.add_argument('--workers', type=int, default=4)

    ## Data generator configuration
    parser.add_argument('--crop_size', type=int, default=256)
    parser.add_argument('--scale', type=int, default=4)
    parser.add_argument('--bands', type=int, default=172)
    parser.add_argument('--msi_bands', type=int, default=4)
    parser.add_argument('--mis_pix', type=int, default=0)
    parser.add_argument('--mixed_align_opt', type=int, default=0)
    parser.add_argument('--student_layers', type=list, default=[2,2,2,2])
    
    # Network architecture configuration
    parser.add_argument("--network_mode", type=int, default=1, help="Training network mode: 0) Single mode, 1) LRHSI+HRMSI, 2) COCNN (LRHSI+HRMSI+CO), Default: 2")     
    parser.add_argument('--num_base_chs', type=int, default=86, help='The number of the channels of the base feature')
    parser.add_argument('--num_blocks', type=int, default=6, help='The number of the repeated blocks in backbone')
    parser.add_argument('--num_agg_feat', type=int, default=172//4, help='the additive feature maps in the block')
    parser.add_argument('--groups', type=int, default=1, help="light version the group value can be >1, groups=1 for full COCNN version, groups=4 is COCNN-Light for 4 HRMSI version")
    
    # Others
    # parser.add_argument("--root", type=str, default="/work/u1657859/DCSNv2/Data", help='data root folder')
    parser.add_argument("--root", type=str, default="/work/u5088463/Data", help='data root folder')   
    parser.add_argument("--test_file", type=str, default="./test.txt")   
    parser.add_argument("--prefix", type=str, default="KD_QRCODE_band4")  
    parser.add_argument("--device", type=str, default="cuda:0", help="cuda:device_id or cpu")  
    parser.add_argument("--DEBUG", type=bool, default=False)  
    parser.add_argument("--gpus", type=int, default=1)
    parser.add_argument("-f", type=str, default=1)
    # parser.add_argument("--test_mode", type=int, default=2 , help='1 : test teacher , 2 : test student' ) 
    
    
    args = parser.parse_args()

    return args

args = parse_args()
m1=Student(args)
m2=Teacher(args)

from thop import profile

input1 = torch.randn(1, 4, 256, 256) 
input2 = torch.randn(1, 172, 64,64)           
flops, params = profile(m1, inputs=(input2,input1, ))
print('FLOPs = ' + str(flops/1000**3) + 'G')
print('Params = ' + str(params/1000**2) + 'M')

flops, params = profile(m2, inputs=(input2,input1, ))
print('FLOPs = ' + str(flops/1000**3) + 'G')
print('Params = ' + str(params/1000**2) + 'M')



[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_relu() for <class 'torch.nn.modules.activation.LeakyReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
FLOPs = 143.245606912G
Params = 7.426272M
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_relu() for <class 'torch.nn.modules.activation.LeakyReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
FLOPs = 940.241010688G
Params = 26.784012M


In [2]:
from PeerMethod.MSSJFL import Net

m3 = Net(172,4,4)
flops, params = profile(m3, inputs=(input2,input1, ))
print('FLOPs = ' + str(flops/1000**3) + 'G')
print('Params = ' + str(params/1000**2) + 'M')

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_relu() for <class 'torch.nn.modules.activation.LeakyReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.AdaptiveMaxPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pixelshuffle.PixelShuffle'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.ConvTranspose2d'>.
FLOPs = 175.565520704G
Params = 16.335277M


In [2]:
import os
import time
import numpy as np
# from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
# from sklearn.metrics import log_loss
import random as rn
# from models.dcsn3d import DCSNMain as DCSN
from models.KQD import Teacher , Student
# from models.dcsn2d_light_MSI_awgn import DCSN_Teacher , DCSN_Student
# from utils import *
import torch
import numpy as np
# from dataset import *
from scipy.io import savemat, loadmat
# from trainOps import *
import tqdm
import time
import argparse

# torch.backends.cudnn.benchmark=True
# Hyperparameters
batch_size = 1
device = 'cuda'
MAX_EP = 16000
BANDS = 172
SIGMA = 0.0    ## Noise free -> SIGMA = 0.0

prefix='DCSN_joint2'
DEBUG=False


def parse_args():
    parser = argparse.ArgumentParser(description='Train Convex-Optimization-Aware SR net')
    
    parser.add_argument('--SEED', type=int, default=1029)
    parser.add_argument('--batch_size', type=int, default=1)

    parser.add_argument('--resume_ind', type=int, default=999)
    parser.add_argument('--snr', type=int, default=0)
    
    
    parser.add_argument('--workers', type=int, default=4)

    ## Data generator configuration
    parser.add_argument('--crop_size', type=int, default=256)
    parser.add_argument('--scale', type=int, default=4)
    parser.add_argument('--bands', type=int, default=172)
    parser.add_argument('--msi_bands', type=int, default=4)
    parser.add_argument('--mis_pix', type=int, default=0)
    parser.add_argument('--mixed_align_opt', type=int, default=0)
    parser.add_argument('--adaptive_fuse', type=int, default=3)
    parser.add_argument('--student_layers', type=list, default=[1,2,2,1])
    
    # Network architecture configuration
    parser.add_argument("--network_mode", type=int, default=1, help="Training network mode: 0) Single mode, 1) LRHSI+HRMSI, 2) COCNN (LRHSI+HRMSI+CO), Default: 2")     
    parser.add_argument('--num_base_chs', type=int, default=86, help='The number of the channels of the base feature')
    parser.add_argument('--num_blocks', type=int, default=6, help='The number of the repeated blocks in backbone')
    parser.add_argument('--num_agg_feat', type=int, default=172//4, help='the additive feature maps in the block')
    parser.add_argument('--groups', type=int, default=1, help="light version the group value can be >1, groups=1 for full COCNN version, groups=4 is COCNN-Light for 4 HRMSI version")
    
    # Others
    # parser.add_argument("--root", type=str, default="/work/u1657859/DCSNv2/Data", help='data root folder')
    parser.add_argument("--root", type=str, default="/work/u5088463/Data", help='data root folder')   
    parser.add_argument("--test_file", type=str, default="./test.txt")   
    parser.add_argument("--prefix", type=str, default="KD_QRCODE_band4")  
    parser.add_argument("--device", type=str, default="cuda:0", help="cuda:device_id or cpu")  
    parser.add_argument("--DEBUG", type=bool, default=False)  
    parser.add_argument("--gpus", type=int, default=1)
    parser.add_argument("-f", type=str, default=1)
    # parser.add_argument("--test_mode", type=int, default=2 , help='1 : test teacher , 2 : test student' ) 
    
    
    args = parser.parse_args()
    args.student_layers = list(map(int, args.student_layers))

    return args

args = parse_args()
m1=Student(args).cuda()
# m2=Teacher(args)
def prepare_input(resolution):
    lr_hsi = torch.FloatTensor(1,172,64,64).cuda()
    hr_msi = torch.FloatTensor(1,4,256,256).cuda()
    return dict(LRHSI =lr_hsi, HRMSI=hr_msi)
        

##==========================================
from ptflops import get_model_complexity_info
device = 'cuda:0'
# model = UNet(172,4).cuda()
flops, params = get_model_complexity_info(m1, input_res=(1, 224, 224), 
                                          input_constructor=prepare_input,
                                          as_strings=True, print_per_layer_stat=False)
print('      - Flops:  ' + flops)
print('      - Params: ' + params)

from thop import profile

LRHSI = torch.FloatTensor(1,172,64,64).cuda()
HRMSI = torch.FloatTensor(1,4,256,256).cuda()        
flops, params = profile(m1, inputs=(LRHSI,HRMSI, ))
print('FLOPs = ' + str(flops/1000**3) + 'G')
print('Params = ' + str(params/1000**2) + 'M')

# flops, params = profile(m2, inputs=(input2,input1, ))
# print('FLOPs = ' + str(flops/1000**3) + 'G')
# print('Params = ' + str(params/1000**2) + 'M')


repetitions = 1024
# LRHSI = torch.FloatTensor(1,172,64,64).cuda()
# HRMSI = torch.FloatTensor(1,4,256,256).cuda()
    
m1=m1.cuda()
print('GPU warm up ...\n')
with torch.no_grad():
    for _ in range(10):
        _ = m1(LRHSI, HRMSI)

# synchronize 等待所有 GPU 任务处理完才返回 CPU 主线程
torch.cuda.synchronize()
# 设置用于测量时间的 cuda Event, 这是PyTorch 官方推荐的接口,理论上应该最靠谱
starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
# 初始化一个时间容器
timings = np.zeros((repetitions, 1))
print('Inference Time Analysis ...\n')
with torch.no_grad():
    for rep in tqdm.tqdm(range(repetitions)):
        starter.record()
        _ = m1(LRHSI, HRMSI)
        ender.record()
        torch.cuda.synchronize() # 等待GPU任务完成
        curr_time = starter.elapsed_time(ender) # 从 starter 到 ender 之间用时,单位为毫秒
        timings[rep] = curr_time
avg = timings.sum()/repetitions
print('\nTotal={} ms'.format(timings.sum()))

print('\nAvg={} ms'.format(avg))

Warning! No positional inputs found for a module, assuming batch size is 1.
      - Flops:  134.07 GMac
      - Params: 6.09 M
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_relu() for <class 'torch.nn.modules.activation.LeakyReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_softmax() for <class 'torch.nn.modules.activation.Softmax'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
FLOPs = 133.898144704G
Params = 4.790482M
GPU warm up ...

Inference Time Analysis ...



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1024/1024 [00:29<00:00, 34.89it/s]


Total=28020.46976661682 ms

Avg=27.36374000646174 ms


In [4]:
import torch
import torch.nn.functional as F
from torch.profiler import profile, record_function, ProfilerActivity
import torch.nn as nn
import torch.nn.init
from ptflops import get_model_complexity_info
import tqdm
from torch.backends import cudnn
cudnn.benchmark = True
import numpy as np

#
class ChannelAttention(nn.Module):
    def __init__(self, in_planes):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc1 = nn.Conv2d(in_planes, in_planes//4, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Conv2d(in_planes//4, in_planes, 1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, in_planes):
        super(SpatialAttention, self).__init__()
        self.fc1 = nn.Conv2d(in_planes, in_planes//2, 3, padding=1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Conv2d(in_planes // 2, 1, 3, padding=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(x)))
        return self.sigmoid(avg_out)


class mrcam(nn.Module):
    def __init__(self, hs_channels,mschannels, bias=True):
        super(mrcam, self).__init__()
        self.conv0 = nn.Sequential(nn.Conv2d(hs_channels+mschannels, 64, kernel_size=3, stride=1, padding=1, bias=bias),

                     nn.LeakyReLU(0.2)
         )

        self.conv0_1 = nn.Parameter(torch.FloatTensor([1]),requires_grad=True)

        self.conv1 = nn.Sequential(

                nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=bias),

        )

        self.conv1_1 = nn.Parameter(torch.FloatTensor([1]),requires_grad=True)

        self.conv2 = nn.Sequential(

                nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=bias),

                nn.LeakyReLU(0.2),

                nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=bias),

        )
        self.conv1_2 = nn.Parameter(torch.FloatTensor([1]),requires_grad=True)

        self.conv3=nn.Sequential(

                nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=bias),

                nn.LeakyReLU(0.2),

                nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=bias),

                nn.LeakyReLU(0.2),

                nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=bias),

        )
        self.conv1_3 = nn.Parameter(torch.FloatTensor([1]),requires_grad=True)

        self.cb1 = nn.Conv2d(64*2, 64, kernel_size=3, stride=1, padding=1, bias=bias)
        self.cb2 = nn.Conv2d(64*3, 64, kernel_size=3, stride=1, padding=1, bias=bias)

        self.cb = nn.Sequential(
            nn.Conv2d(64*3, 64, kernel_size=3, stride=1, padding=1, bias=bias),
            nn.LeakyReLU(0.2)
        )

        self.channel_3_3 = ChannelAttention(64)
        self.spatial_3_3 = SpatialAttention(64)
        self.factor_3 = nn.Parameter(torch.FloatTensor([1]),requires_grad=True)
        self.channel_5_5 = ChannelAttention(64)
        self.spatial_5_5 = SpatialAttention(64)
        self.factor_5 = nn.Parameter(torch.FloatTensor([1]),requires_grad=True)
        self.channel_7_7 = ChannelAttention(64)
        self.spatial_7_7 = SpatialAttention(64)
        self.factor_7 = nn.Parameter(torch.FloatTensor([1]),requires_grad=True)
        self.fusion = nn.Sequential(

                nn.Conv2d(64*3, 64, kernel_size=1, stride=1, padding=0, bias=bias),

                nn.LeakyReLU(0.2),

                nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=bias),

        )

    def forward(self, x):
        x0 = x = self.conv0(x)
        x1 = self.conv1(x)
        x2 = self.conv2(x1)
        x3 = self.conv3(x2)
        cb1 = torch.cat((x1*self.conv1_1, x2*self.conv1_2), dim=1)
        cb2 = torch.cat((x1*self.conv1_1, x2*self.conv1_2, x3*self.conv1_3), dim=1)

        cb1 = self.cb1(cb1)
        cb2 = self.cb2(cb2)
        cb3 = torch.cat((x1*self.conv1_1,cb1,cb2),dim=1)
        x = self.cb(cb3)

        atten1 = self.channel_3_3(x)*self.spatial_3_3(x)*x + x
        atten2 = self.channel_5_5(x)*self.spatial_5_5(x)*x + x
        atten3 = self.channel_7_7(x)*self.spatial_7_7(x)*x + x
        atten1 = self.factor_3*atten1
        atten2 = self.factor_5*atten2
        atten3 = self.factor_7*atten3

        atten = torch.cat((atten1, atten2, atten3), dim=1)
        x = self.fusion(atten) + x0
        return x


class UNet(nn.Module):
    def __init__(self, hschannels, mschannels, bilinear=True):
        super(UNet, self).__init__()
        self.bilinear = bilinear
        factor = 2 if bilinear else 1
        #hsi 主网络
        self.inc = DoubleConv(64, 64)
        self.mscsam = mrcam(hschannels, mschannels)
        self.down1 = Down(64+32, 128)
        self.down2 = Down(128+64, 256)
        self.down3 = Down(256+128, 512 // factor)
        # self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(256+(512 // factor)+256// factor, 256 // factor, 256, bilinear)
        self.up2 = Up(128+256 // factor+128// factor, 128 // factor, 128, bilinear)
        self.up3 = Up(64+128 // factor+64// factor, 64, 64, bilinear)
        # self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, hschannels)
        #ms特征提取网络
        self.incm = DoubleConv(mschannels, 32)
        self.downm1 = Down(32, 64)
        self.downm2 = Down(64, 128)
        self.downm3 = Down(128, 256 // factor)
        # self.down4 = Down(512, 1024 // factor)
        # self.upm1 = Up(256, 128 // factor, bilinear)
        # self.upm2 = Up(128, 64 // factor, bilinear)
        # self.upm3 = Up(64, 32, bilinear)
        self.upm1 = Upm(256 // factor, 128 // factor, bilinear)
        self.upm2 = Upm(128 // factor, 64 // factor, bilinear)
        self.upm3 = Upm(64 // factor, 64, bilinear)
        # self.up4 = Up(128, 64, bilinear)
        self.outmc = OutConv(64, hschannels)

    def forward(self, zz):
        y0=zz[1]
        y1 = self.incm(zz[1])
        y2 = self.downm1(y1)
        y3 = self.downm2(y2)
        y4 = self.downm3(y3)
        # y5 = self.upm1(y4, y3)
        # y6 = self.upm2(y5, y2)
        # y = self.upm3(y6, y1)
        y5 = self.upm1(y4)
        y6 = self.upm2(y5)
        y = self.upm3(y6)
        logitsy = self.outmc(y)

        x0 = zz[0]
        # 嵌入多尺度注意力
        # x1 = self.mscsam(x)

        x1 = self.mscsam(torch.cat((zz[0],y0),dim=1))
        x1 = self.inc(x1)

        x2 = self.down1(torch.cat((x1, y1), dim=1))
        x3 = self.down2(torch.cat((x2, y2), dim=1))
        x4 = self.down3(torch.cat((x3, y3), dim=1))
        # x5 = self.down4(x4)
        x = self.up1(torch.cat((x4, y4),dim=1),x3)
        x = self.up2(torch.cat((x, y5),dim=1),x2)
        x = self.up3(torch.cat((x, y6),dim=1),x1)
        # x = self.up4(x, x1)
        logits = self.outc(x)
        # return logits+x0, logits+x0

        return logits+x0, logits+x0+logitsy


class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.LeakyReLU(0.2),

        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.Upsample(mode='bilinear',scale_factor=1/2),
            DoubleConv(in_channels, out_channels)
            # nn.MaxPool2d(2),
            # DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels-channels, (in_channels-channels) // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv((in_channels-channels) // 2+channels, out_channels)


    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)
class Upm(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels // 2, out_channels)


    def forward(self, x1):
        x1 = self.up(x1)
        return self.conv(x1)

class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size=3,stride=1,padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(in_channels, out_channels, kernel_size=1),

        )
    def forward(self, x):
        return self.conv(x)

def prepare_input(resolution):
    lr_hsi = torch.FloatTensor(1,172,256,256).cuda()
    hr_msi = torch.FloatTensor(1,4,256,256).cuda()
    return dict(zz = [lr_hsi,hr_msi])
        

##==========================================

if __name__ == '__main__':
    
    #Flops & Parameter
    # Create a network and a corresponding input
    device = 'cuda:0'
    model = UNet(172,4).cuda()
    flops, params = get_model_complexity_info(model, input_res=(1, 224, 224), 
                                              input_constructor=prepare_input,
                                              as_strings=True, print_per_layer_stat=False)
    print('      - Flops:  ' + flops)
    print('      - Params: ' + params)
    
    LRHSI = torch.FloatTensor(1,172,256,256).cuda()
    HRMSI = torch.FloatTensor(1,4,256,256).cuda()
    zz = [LRHSI,HRMSI]
    y1,y2=model(zz)
    print(y1.shape, y2.shape)
  

    #Use Profile
    #with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    #    with record_function("model_inference"):
    #        model(zz)
    #print(prof.key_averages().table(sort_by="cuda_time_total"))

    #Use EndTime - StartTime with GPU synchronize
    repetitions = 1024
    print('GPU warm up ...\n')
    with torch.no_grad():
        for _ in range(10):
            _ = model(zz)
    
    # synchronize 等待所有 GPU 任务处理完才返回 CPU 主线程
    torch.cuda.synchronize()
    # 设置用于测量时间的 cuda Event, 这是PyTorch 官方推荐的接口,理论上应该最靠谱
    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    # 初始化一个时间容器
    timings = np.zeros((repetitions, 1))
    print('Inference Time Analysis ...\n')
    with torch.no_grad():
        for rep in tqdm.tqdm(range(repetitions)):
            starter.record()
            _ = model(zz)
            ender.record()
            torch.cuda.synchronize() # 等待GPU任务完成
            curr_time = starter.elapsed_time(ender) # 从 starter 到 ender 之间用时,单位为毫秒
            timings[rep] = curr_time
    avg = timings.sum()/repetitions
    print('\nTotal={} ms'.format(timings.sum()))
    
    print('\nAvg={} ms'.format(avg))

Warning! No positional inputs found for a module, assuming batch size is 1.
      - Flops:  102.04 GMac
      - Params: 7.33 M
torch.Size([1, 172, 256, 256]) torch.Size([1, 172, 256, 256])
GPU warm up ...

Inference Time Analysis ...



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1024/1024 [00:38<00:00, 26.52it/s]


Total=36354.67979621887 ms

Avg=35.50261698849499 ms
